# Revisión de Flujos de potencia de elementos en casos de PSS/E

## Objetivo.
En este notebook se obtiene un reporte de flujos de potencia a través de las líneas de transmisión y transformadores que se especifiquen. La explotación de los datos se hace para el total de casos de PSS/E (_*.sav_) que se encuentran en un directorio.

## Problema.
Se requiere obtener el flujo en MW de los enlaces mostrados en la siguiente figura (flechas verdes). La extracción de los valores de flujos se hará de todos los casos que se encuentran en una misma carpeta y corresponden a los escenarios de verano de 2023.

<img src="images\NB-Flujos por elementos-Figura 1.png">

**Los casos originales los puedes descargar de la siguiente liga:**

[Casos 2023 (verano 16 Horas) 26-Ago-2022](https://cenacegobmx.sharepoint.com/:f:/r/sites/PAM2018-20332/Documentos%20compartidos/PSSE/2023_16Hverano?csf=1&web=1&e=uaWdCy)

**La tabla de resultados que se quiere obtener es de la siguiente forma:**

| Casos .sav | Flujo RAP - DMD | Flujo RAP - SLR | Flujo TR SAL | Flujo TR DMD |
| ---    | --- | --- | --- | --- |
| caso 1 | 650 | 648 | 85  | 147 |
| caso 2 | 650 | 648 | 85  | 147 |
| ...    | ... | ... | ... | ... |
| caso n | 650 | 648 | 85  | 147 |

## Otras consideraciones
El caso de 2023 tiene considerada la obra del segundo entronque del Derramadero (P18-NE2), pero de acuerdo con lo que se pide, el flujo de potencia es con la topología anterior (sin considerar la obra), por lo que,antes de obtener el reporte de flujos, se requiere correr un _script .py_ que haga esa modificación y resuleva flujos de potencia considerando el cambio.

# Solución

### Importar librerías e instanciar el PSS/E

In [1]:
#from lib_psse39 import * # Para intérprete de Python 3.9 
from lib_psse37 import * # Para intérprete de Python 3.7 
from src.funcs35 import *

    Sets PSSE environment to latest minor version among installed versions of PSSE 35.
    Use psse35.set_minor(n) to set PSSE35 minor version (n) to use.
        Example, for PSSE 35.0.x, use this as: psse35.set_minor(0)

 Input error detected at !
 -m ipykernel_launcher --ip=127.0.0.1 --stdin=9044 --control=9042 --hb=9041 "--Session.signature_scheme=\"hmac-sha256\"" "--Session.key=b\"9b717dfa-d06c-4777-8ef3-9346f29cc421\"" --shell=9043 "--transport=\"tcp\"" --iopu
                                                                                                          !

 PSS(R)E Version 35
 Copyright (c) 1976-2022
 Siemens Industry, Inc.,
 Power Technologies International                            (PTI)
 This program is a confidential  unpublished  work  created  and  first
 licensed in 1976.  It is a trade secret which is the property of  PTI.
 All use,  disclosure,  and/or reproduction not specifically authorized
 by  PTI  is prohibited.   This  program is protected  under  copyr

### Leer los casos y realizar la corrección de topología (quitar la obra del PEM P18-NE2)

**NOTA: El código de las siguientes dos celdas solo se debería ejecutar si el PEM P18-NE2 se encuentra modelado en los casos.**

In [ ]:
CASOS_PATH = "./casos/2023_20Hinvierno/" # Ruta donde se guardan los archivos .sav (de acuerdo con la estructura del repositorio)
casos_sav = os.listdir(CASOS_PATH) # Genera una lista con los nombres de los casos

In [ ]:
# Se instancian las variables _i, _f y _s que son utilizadas para obtener los valores que trae por defecto o que no quieren cambiarse

_i = psspy.getdefaultint()
_f = psspy.getdefaultreal()
_s = psspy.getdefaultchar()

for casos in casos_sav:
    # Abre un caso a la vez
    psspy.case(os.path.abspath("casos/2023_20Hinvierno/" + casos))
    # Hace las modificaciones correspondientes a eliminar el entronque en Derramadero y modelar la línea original (RAP - SLR)
    psspy.purgbrn(640004,640099,r"""ET""")
    psspy.purgbrn(32277,640099,r"""ET""")
    psspy.moveshunt(640099,r"""TL""",640004,r"""TL""")
    psspy.branch_data_3(640004,32277,r"""1""",[1,640004,640,0,0,0],[0.0, 0.0001,0.0,0.0,0.0,0.0,0.0,0.0, 1.0, 1.0, 1.0, 1.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"")
    psspy.branch_chng_3(640004,32277,r"""1""",[_i,_i,_i,_i,_i,_i],[ 0.00261, 0.03225, 0.95188,_f,_f,_f,_f, 136.7,_f,_f,_f,_f],[ 1385.6, 1524.2, 1538.1,_f,_f,_f,_f,_f,_f,_f,_f,_f],_s)
    psspy.seq_branch_data_3(640004,32277,r"""1""",_i,[ 0.02479, 0.10282, 0.56678,_f,_f,_f,_f,_f])
    psspy.mbidbrn(640004,32277,r"""1""",r"""G1""")
    # Se resuleve el caso de flujos de potencia
    psspy.fnsl([0,0,0,1,2,1,99,0])
    # Guardamos el caso en la misma ruta
    psspy.save(os.path.abspath("casos/2023_20Hinvierno/" + casos))

In [2]:
CASOS_PATH = "./casos/2023_20Hinvierno/" # Ruta donde se guardan los archivos .sav (de acuerdo con la estructura del repositorio)
casos_sav = os.listdir(CASOS_PATH) # Genera una lista con los nombres de los casos


In [3]:
ELEM_NOM = ['RAP - DMD',
           'RAP - SLR',
           'TR-1 SAL',
           'TR-2 SAL',
           'TR-1 RAP']

ELEM = ['640004-32277-G1',
        '640004-640099-5D',
        '640003-02',
        '640003-03',
        '640004-01'] #,'4455-44567-ER']

In [4]:
caso_dic = {}
for caso in casos_sav:
    psspy.case(os.path.abspath("casos/2023_20Hinvierno/" + caso))
    d = dic_flujos(0,[400,400],[3,6])
    FL = []
    for linea in ELEM:
        try:
            FL.append(d[linea][0])
        except:
            print("Enlace ", linea, "no encontrado en el caso o subsistema")
    caso_dic[caso] = FL




 The Saved Case in file d:\CENACE\PSSE\casos\2023_20Hinvierno\5-C15_2027_PRIFS20H_OCC_NTE_NES - copia (2).sav was saved on SAT, AUG 27 2022  13:02



 The Saved Case in file d:\CENACE\PSSE\casos\2023_20Hinvierno\5-C15_2027_PRIFS20H_OCC_NTE_NES - copia.sav was saved on SAT, AUG 27 2022  13:02



 The Saved Case in file d:\CENACE\PSSE\casos\2023_20Hinvierno\5-C15_2027_PRIFS20H_OCC_NTE_NES.sav was saved on SAT, AUG 27 2022  13:02


In [10]:
resultado = pd.DataFrame(caso_dic).transpose()
resultado.set_axis(ELEM_NOM, axis=1, inplace=True)
resultado

,RAP - DMD,RAP - SLR,TR-1 SAL,TR-2 SAL,TR-1 RAP
5-C15_2027_PRIFS20H_OCC_NTE_NES - copia (2).sav,539.560791,518.914062,51.319546,51.319546,224.8909
5-C15_2027_PRIFS20H_OCC_NTE_NES - copia.sav,539.560791,518.914062,51.319546,51.319546,224.8909
5-C15_2027_PRIFS20H_OCC_NTE_NES.sav,539.560791,518.914062,51.319546,51.319546,224.8909
